In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor


C:\Users\hikim.TPR\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#required functions

#locating non-numeric values so that I can replace them as NaN value 
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

#removing null rows in specfic column(s) from a df
def rm_null(df, i):
    return df[df.iloc[:,i].isnull()]

In [3]:
##I just like to attach column names
hedgecolumns=pd.read_csv("columnnames.csv", encoding =  "ISO-8859-1")
hcol=hedgecolumns.values.T.tolist()[0]

##reading data.csv
hedgeable=pd.read_csv("data_orginal.csv", names=hcol, low_memory=False, skipinitialspace=True)#, skipinitialspace=True, na_values=["?"])
print(len(hedgeable))


3279


In [4]:
#########Dealing with null values#########

#First, I noticed the null values have string that involves "?"
#I replace non-numeric values with NaN in all attribute values
hedgeable.iloc[:,:-1]=hedgeable.iloc[:,:-1][hedgeable.iloc[:,:-1].applymap(isnumber)]

#The instruction stated that "28% of instances are missing some of the continuous attributes."
#However, one binary column also had 15 null values. 15 out of 3,279 is a small number that I wanted to simply omit.
#I simply omit NaN values in the binary attribute columns & response variable column

hedgeable=hedgeable.dropna(subset=hedgeable.columns[3:-1], how='any')
len(hedgeable)


3264

In [6]:
##28% of instances are missing some of the continuous attributes.
##Too big to simply omit in this case. (will try later anyways)
##So, I'm gonna RF regress for the missing data

##hedgeable rows without na values
hedgeclean=hedgeable.dropna()
##hedgeable rows only with na values
hedgena=hedgeable[hedgeable.isnull().any(axis=1)]


#getting rows where each column has null values. So I can replace these with predicted values


height=rm_null(hedgeable, 0)    
width =rm_null(hedgeable, 1)  
aratio=rm_null(hedgeable, 2)  

print(len(height))
print(len(width))
print(len(aratio))

898
896
905


In [9]:
#To fill NaN values of the three continuous variables, I am predicting theses values using random forest
#height and weight could use both RF regressor & Classifier
#The minimum number of groups for any class cannot be less than n_splits=3
#So I am trying regressors: svm and rf 

#print(len(df_width_cl.value_counts())) #219
#print(len(df_width_cl.value_counts())) #277

#predictor
df_x_cl=hedgeclean.iloc[:,3:-1] #2359 rows × 1555 columns

#response
df_height_cl=hedgeclean.iloc[:,0] #2359 rows × 1 columns
df_width_cl=hedgeclean.iloc[:,1] #2359 rows × 1 columns
df_aratio_cl=hedgeclean.iloc[:,2] #2359 rows × 1 columns

#splitting test vs train
x_train_ht, x_test_ht, y_train_ht, y_test_ht = train_test_split(df_x_cl, df_height_cl, test_size=0.2, random_state=1)
x_train_wdt, x_test_wdt, y_train_wdt, y_test_wdt = train_test_split(df_x_cl, df_width_cl, test_size=0.2, random_state=1)
x_train_rt, x_test_rt, y_train_rt, y_test_rt = train_test_split(df_x_cl, df_aratio_cl, test_size=0.2, random_state=1)

In [12]:
print(len(hedgeclean.iloc[:,0].value_counts()))
print(len(hedgeclean.iloc[:,1].value_counts()))

219
277


In [18]:
#RF cv over tree sizes


for i in range(40, 300, 30):
    rf_reg=RandomForestRegressor(n_estimators=i)
    cv = cross_validation.KFold(len(df_x_cl), n_folds=7)
    regressor_scores = cross_val_score(rf_reg, x_train_ht, y_train_ht, scoring='neg_mean_squared_error')
    print("rf_regressor_score: ",i," ", regressor_scores.mean())

rf_regressor_score:  40   -1852.40806563
rf_regressor_score:  70   -1846.57603409
rf_regressor_score:  100   -1810.83914598
rf_regressor_score:  130   -1811.79446615
rf_regressor_score:  160   -1829.4672157
rf_regressor_score:  190   -1833.32885974
rf_regressor_score:  220   -1834.42773338
rf_regressor_score:  250   -1822.56990029
rf_regressor_score:  280   -1815.74145804


In [ ]:
sv_reg=svm.SVR()(n_estimators=100)
cv = cross_validation.KFold(len(df_x_cl), n_folds=7)
regressor_scores = cross_val_score(sv_reg, x_train_ht, y_train_ht, scoring='neg_mean_squared_error')
print("svr_score: ",i," ", regressor_scores.mean())

In [11]:
#model types
#rf=RandomForestClassifier(n_estimators=50)
#svc=svm.SVC()

svr=svm.SVR()

#cv


#cv 
regressor_scores = cross_val_score(rf_reg, x_train_ht, y_train_ht)
svm_scores = cross_val_score(svr, x_train_ht, y_train_ht)
#print("classifier score: ", classifier_scores.mean())
print("rf_regressor_score: ", regressor_scores.mean())
#print("svm_score: ", svm_scores.mean()) 

rf_regressor_score:  0.371502747756


In [19]:
rf_reg=RandomForestRegressor(n_estimators=100)


#applying fit model to the test set
rf_reg.fit(x_test_ht, y_test_ht)
svr.fit(x_test_ht,y_test_ht)

#preditcted values
rf_pred_ht_rf=rf_reg.predict(x_test_ht)
rf_pred_ht_svr=svr.predict(x_test_ht)
s=y_test_ht.tolist()

rf_squarederror=[]
sv_squarederror=[]
for i in range(len(rf_pred_ht_rf)):
    rf_squarederror.append(math.sqrt(((rf_pred_ht_rf[i]-float(s[i]))**2)))
    sv_squarederror.append(math.sqrt(((rf_pred_ht_svr[i]-float(s[i]))**2)))


print(sum(rf_squarederror)/len(rf_squarederror))
print(np.mean(rf_squarederror))
print(np.std(rf_squarederror))

#pd.Series(list(s))
print(sum(sv_squarederror)/len(sv_squarederror))
print(np.mean(sv_squarederror))
print(np.std(sv_squarederror))

11.891891659664108
11.8918916597
19.676368353
39.65887586289224
39.6588758629
47.7784336646


In [20]:
height_x=height.iloc[:,3:-1]
width_x=width.iloc[:,3:-1]
aratio_x=aratio.iloc[:,3:-1]

rf_reg.fit(df_x_cl, df_height_cl)

height_pred=rf_reg.predict(height_x)
#width_pred=rf_reg.predict(width_x)
#aratio_pred=rf_reg.predict(aratio_x)

In [100]:
print(len(height_pred))
print(len(y_test_ht.values))

898
472


12.044862087136268
12.0448620871
19.6251025719
0.0


In [99]:
#replacing NaN values with the predicted values
hedgeable.loc[hedgeable['height: continuous.'].isnull(), 'height: continuous.']=list(height_pred)
hedgeable.loc[hedgeable['width: continuous.'].isnull(), 'width: continuous.']=list(width_pred)
hedgeable.loc[hedgeable['aratio: continuous.'].isnull(), 'aratio: continuous.']=list(aratio_pred)


In [100]:
#making sure there's no null values
print(len(hedgeable))
print(len(hedgeable.dropna()))

3264
3264


In [122]:
x=hedgeable.iloc[:,:-1]
ad=hedgeable.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, ad, test_size=0.3)#, random_state=3)

#classifying now 
rf=RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)
pred=rf.predict(x_test)


s=y_test.values

squarederror=[]
correct=0
for i in range(len(pred)):
    if pred[i]==s[i]:
        correct=correct+1
    else:
        correct=correct+0

print(correct/len(s))

0.976530612244898


In [118]:
s=y_test.values

squarederror=[]
correct=0
for i in range(len(pred)):
    if pred[i]==s[i]:
        correct=correct+1
    else:
        correct=correct+0

print(correct/len(s))

0.9806122448979592
